# Data science workflow: From csv to machine learning

Author : Laurent Morelli

The objective of this lecture to present and discuss the standard
issues one have to deal with when working with actual data:

   - presence of missing values (the imputation problem)
   - presence of a mix of continuous and categorical variables
   - complex (non normal) marginal distributions with potential outliers

In this lecture we'll use the pandas package (http://pandas.pydata.org).

The data we'll manipulate is from the Titanic kaggle challenge.

https://www.kaggle.com/c/titanic-gettingStarted

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Loading tabular data from the Titanic kaggle challenge in a pandas Data Frame

You can download the data from:

https://dl.dropboxusercontent.com/u/2140486/data/titanic_train.csv

Let's load the CSV file as a pandas data frame in one line:

In [ ]:
#!curl -s https://dl.dropboxusercontent.com/u/2140486/data/titanic_train.csv | head -5
!head -5 /var/www/python-4-TP_datascience_workflow_pandas-matters/train.csv

In [ ]:
#data = pd.read_csv('https://dl.dropboxusercontent.com/u/2140486/data/titanic_train.csv')
data = pd.read_csv('/var/www/python-4-TP_datascience_workflow_pandas-matters/train.csv')

The `data` object is an instance pandas `DataFrame` class:

In [ ]:
print(type(data))

pandas data frames have a HTML table representation in the IPython notebook. Let's have a look at the first 5 rows:

In [ ]:
data.head(5)

In [ ]:
data.count()

The data frame has 891 rows. Some passengers have missing information though: in particular Age and Cabin info can be missing. The meaning of the columns is explained on the challenge website:

https://www.kaggle.com/c/titanic-gettingStarted/data

and copied here:

```
VARIABLE DESCRIPTIONS:
survival        Survival
                (0 = No; 1 = Yes)
pclass          Passenger Class
                (1 = 1st; 2 = 2nd; 3 = 3rd)
name            Name
sex             Sex
age             Age
sibsp           Number of Siblings/Spouses Aboard
parch           Number of Parents/Children Aboard
ticket          Ticket Number
fare            Passenger Fare
cabin           Cabin
embarked        Port of Embarkation
                (C = Cherbourg; Q = Queenstown; S = Southampton)

SPECIAL NOTES:
Pclass is a proxy for socio-economic status (SES)
 1st ~ Upper; 2nd ~ Middle; 3rd ~ Lower

Age is in Years; Fractional if Age less than One (1)
 If the Age is Estimated, it is in the form xx.5

With respect to the family relation variables (i.e. sibsp and parch)
some relations were ignored.  The following are the definitions used
for sibsp and parch.

Sibling:  Brother, Sister, Stepbrother, or Stepsister of Passenger Aboard Titanic
Spouse:   Husband or Wife of Passenger Aboard Titanic (Mistresses and Fiances Ignored)
Parent:   Mother or Father of Passenger Aboard Titanic
Child:    Son, Daughter, Stepson, or Stepdaughter of Passenger Aboard Titanic

Other family relatives excluded from this study include cousins,
nephews/nieces, aunts/uncles, and in-laws.  Some children travelled
only with a nanny, therefore parch=0 for them.  As well, some
travelled with very close friends or neighbors in a village, however,
the definitions do not support such relations.
```

### What can you do with a Pandas DataFrame?

In [ ]:
list(data.columns)  # get the names of the columns

In [ ]:
data.shape  # get the shape

In [ ]:
data.values  # get the content as a numpy array

In [ ]:
data.dtypes  # get the data type (dtype) of each column

### Problem: one can not directly learn from such data


- the target variable (survival) is mixed with the input data

- some attributes such as unique ids have no predictive values for the task

- the values are heterogeneous with dtype ``object`` (string labels for categories, integers and floating point numbers)

- some attribute values are missing (nan: "not a number")

## Predicting survival

The goal is to predict whether a passenger has survived from others known attribute. Let us have a look at the `Survived` columns:

In [ ]:
survived_column = data['Survived']  # or data.Survived

`data.Survived` is an instance of the pandas `Series` class with an integer dtype:

In [ ]:
type(survived_column)

In [ ]:
survived_column.dtype

`Series` can be seen as **homegeneous**, 1D columns. `DataFrame` instances are **heterogenous collections of columns with the same length**.

The original data frame can be aggregated by counting rows for each possible value of the `Survived` column:

In [ ]:
data.groupby('Survived').count()

In [ ]:
np.mean(survived_column == 0)

### Question:

- Is the dataset balanced? What will be the chance level accuracy?
- What variables contain more missing values?
- Is the absence of a variable predictive?

From this the subset of the full passengers list, about 2/3 perished in the event. So if we are to build a predictive model from this data, a baseline model to compare the performance to would be to always predict death. Such a constant model would reach around 62% predictive accuracy (which is higher than predicting at random):

pandas `Series` instances can be converted to regular 1D numpy arrays by using the `values` attribute:

In [ ]:
target = survived_column.values

In [ ]:
type(target)

In [ ]:
target.dtype

In [ ]:
target[:5]

## Basic visualization

Full doc at http://pandas.pydata.org/pandas-docs/stable/visualization.html

See also seaborn package.

https://stanford.edu/~mwaskom/software/seaborn/

In [ ]:
data.plot(kind='scatter', x='Age', y='Fare', c='Survived', s=50, cmap='viridis');

In [ ]:
from pandas.tools.plotting import scatter_matrix
scatter_matrix(data.get(['Fare', 'Pclass', 'Age']), alpha=0.2,
               figsize=(8, 8), diagonal='kde');

In [ ]:
data.get(['Fare', 'Age', 'Survived']).groupby('Survived').mean().plot(kind='bar');

What we can see is that the 'Fare' variable as a positive and heavy tail distribution.

In [ ]:
data.Fare.plot(kind='hist', bins=20);

Let's try to standardize the distribution

In [ ]:
fare = data.Fare.values
plt.hist((fare - np.mean(fare)) / np.std(fare));

but it does not look very normally distributed. Let's try to apply a non-linear transform to the feature to obtain a marginal that looks quite Gaussian:

In [ ]:
fare = np.log(data.Fare.values + 10.)
plt.hist((fare - np.mean(fare)) / np.std(fare));

In [ ]:
data_plot = data.get(['Age', 'Survived'])
data_plot = data.assign(LogFare=lambda x : np.log(x.Fare + 10.))
scatter_matrix(data_plot.get(['Age', 'LogFare']), alpha=0.2, figsize=(8, 8), diagonal='kde');

In [ ]:
data_plot.plot(kind='scatter', x='Age', y='LogFare', c='Survived', s=50, cmap='viridis');

### Question:

- Can you get some insights about the data by looking at this graph?

## Training a predictive model on numerical features

Machine learning algorithms as implemented in Scikit-Learn work with homegeneous numerical feature descriptors passed as a numpy array. Therefore passing the raw data frame will not work out of the box.

Let us start simple and build a first model that only uses readily available numerical features as input, namely `data.Fare`, `data.Pclass` and `data.Age`.

In [ ]:
numerical_features = data.get(['Fare', 'Pclass', 'Age'])
print(numerical_features.dtypes)
numerical_features.head(5)

Unfortunately some passengers do not have age information:

In [ ]:
numerical_features.count()

Let's use pandas `fillna` method to input the median age for those passengers:

In [ ]:
median_features = numerical_features.dropna().median()
median_features

In [ ]:
imputed_features = numerical_features.fillna(median_features)
imputed_features.count()

In [ ]:
imputed_features.describe()  # look at some meaningful info about our data

Now that the data frame is clean, we can convert it into an homogeneous numpy array of floating point values:

In [ ]:
features_array = imputed_features.values
features_array

In [ ]:
features_array.dtype

Let's take the 80% of the data for training a first model and keep 20% for computing is generalization score:

In [ ]:
from sklearn.model_selection import train_test_split
# replace model_selection by cross_validation for old version of scikit-learn

features_train, features_test, target_train, target_test = train_test_split(
    features_array, target, test_size=0.20, random_state=0)

In [ ]:
features_train.shape

In [ ]:
features_test.shape

In [ ]:
target_train.shape

In [ ]:
target_test.shape

Let's start with a simple model from sklearn, namely `LogisticRegression`:

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(C=1.)
logreg.fit(features_train, target_train)

In [ ]:
target_predicted = logreg.predict(features_test)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(target_test, target_predicted)

This first model has around 73% accuracy: this is better than our baseline that always predicts death.

In [ ]:
logreg.score(features_test, target_test)

## Model evaluation and interpretation

### Interpreting linear model weights

The `coef_` attribute of a fitted linear model such as `LogisticRegression` holds the weights of each features:

In [ ]:
feature_names = numerical_features.columns
feature_names

In [ ]:
logreg.fit(features_array, target)
logreg.coef_

In [ ]:
x = np.arange(len(feature_names))
plt.bar(x, logreg.coef_.ravel())
_ = plt.xticks(x + 0.5, feature_names, rotation=30)

In this case, survival is slightly positively linked with Fare (the higher the fare, the higher the likelyhood the model will predict survival) while passenger from first class and lower ages are predicted to survive more often than older people from the 3rd class.

First-class cabins were closer to the lifeboats and children and women reportedly had the priority. Our model seems to capture that historical data. We will see later if the sex of the passenger can be used as an informative predictor to increase the predictive accuracy of the model.

### Alternative evaluation metrics

It is possible to see the details of the false positive and false negative errors by computing the confusion matrix:

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(target_test, target_predicted)
print(cm)

The true labeling are seen as the rows and the predicted labels are the columns:

In [ ]:
def plot_confusion(cm):
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.binary)
    plt.title('Confusion matrix')
    plt.set_cmap('Blues')
    plt.colorbar()

    target_names = ['not survived', 'survived']

    tick_marks = np.arange(len(target_names))
    plt.xticks(tick_marks, target_names, rotation=60)
    plt.yticks(tick_marks, target_names)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    # Convenience function to adjust plot parameters for a clear layout.
    plt.tight_layout()
    
plot_confusion(cm)

In [ ]:
print(cm)

We can normalize the number of prediction by dividing by the total number of true "survived" and "not survived" to compute false and true positive rates for survival (in the second column of the confusion matrix).

In [ ]:
print(cm.astype(np.float64) / cm.sum(axis=1))

We can therefore observe that the fact that the target classes are not balanced in the dataset makes the accuracy score not very informative.

scikit-learn provides alternative classification metrics to evaluate models performance on imbalanced data such as precision, recall and f1 score:

In [ ]:
from sklearn.metrics import precision_score, recall_score
precision_score?

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(target_test, target_predicted,
                            target_names=['not survived', 'survived']))

Another way to quantify the quality of a binary classifier on imbalanced data is to compute the precision, recall and f1-score of a model (at the default fixed decision threshold of 0.5).

Logistic Regression is a probabilistic models: instead of just predicting a binary outcome (survived or not) given the input features it can also estimates the posterior probability of the outcome given the input features using the `predict_proba` method:

In [ ]:
target_predicted_proba = logreg.predict_proba(features_test)
target_predicted_proba[:5]

### Cross-validation

We previously decided to randomly split the data to evaluate the model on 20% of held-out data. However the location randomness of the split might have a significant impact in the estimated accuracy:

In [ ]:
features_train, features_test, target_train, target_test = train_test_split(
    features_array, target, test_size=0.20, random_state=0)

logreg.fit(features_train, target_train).score(features_test, target_test)

In [ ]:
features_train, features_test, target_train, target_test = train_test_split(
    features_array, target, test_size=0.20, random_state=1)

logreg.fit(features_train, target_train).score(features_test, target_test)

In [ ]:
features_train, features_test, target_train, target_test = train_test_split(
    features_array, target, test_size=0.20, random_state=2)

logreg.fit(features_train, target_train).score(features_test, target_test)

So instead of using a single train / test split, we can use a group of them and compute the min, max and mean scores as an estimation of the real test score while not underestimating the variability:

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(logreg, features_array, target, cv=5)
scores

In [ ]:
scores.min(), scores.mean(), scores.max(), scores.std()

**Note:** See that we compute the standard deviation of CV scores. It gives you an idea of the variability of the CV scores across folds.

`cross_val_score` reports accuracy by default be it can also be used to report other performance metrics such as ROC-AUC or f1-score:

In [ ]:
scores = cross_val_score(logreg, features_array, target, cv=5,
                         scoring='roc_auc')
scores.min(), scores.mean(), scores.max(), scores.std()

## More feature engineering and richer models

Let us now try to build richer models by including more features as potential predictors for our model.

Categorical variables such as `data.Embarked` or `data.Sex` can be converted as boolean indicators features also known as dummy variables or one-hot-encoded features:

In [ ]:
pd.get_dummies(data.Sex, prefix='Sex').head(5)

In [ ]:
pd.get_dummies(data.Embarked, prefix='Embarked').head(5)

We can combine those new numerical features with the previous features using `pandas.concat` along `axis=1`:

In [ ]:
rich_features = pd.concat([data.get(['Fare', 'Pclass', 'Age']),
                           pd.get_dummies(data.Sex, prefix='Sex'),
                           pd.get_dummies(data.Embarked, prefix='Embarked')],
                          axis=1)
rich_features.head(5)

By construction the new `Sex_male` feature is redundant with `Sex_female`. Let us drop it:

In [ ]:
rich_features_no_male = rich_features.drop('Sex_male', 1)
rich_features_no_male.head(5)

Let us not forget to impute the median age for passengers without age information:

In [ ]:
rich_features_no_male.count()

In [ ]:
rich_features_final = rich_features_no_male.fillna(rich_features_no_male.dropna().median())
rich_features_final.count()

We can finally cross-validate a logistic regression model on this new data an observe that the mean score has significantly increased:

In [ ]:
%%time

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

logreg = LogisticRegression(C=1.)
scores = cross_val_score(logreg, rich_features_final, target, cv=5, scoring='accuracy')
print("Logistic Regression CV scores:")
print("min: {:.3f}, mean: {:.3f}, max: {:.3f}, std: {:.3f}".format(
    scores.min(), scores.mean(), scores.max(), scores.std()))

In [ ]:
feature_names = rich_features_final.columns
logreg.fit(rich_features_final, target)
x = np.arange(len(feature_names))
plt.bar(x, logreg.coef_.ravel())
_ = plt.xticks(x + 0.5, feature_names, rotation=30)

**Exercise**:

- change the value of the parameter `C`. Does it have an impact on the score?

- fit a new instance of the logistic regression model on the full dataset.

- plot the weights for the features of this newly fitted logistic regression model.

## Avoiding data snooping with pipelines

When doing imputation in pandas, prior to computing the train test split we use data from the test to improve the accuracy of the median value that we impute on the training set. This is actually cheating. To avoid this we should compute the median of the features on the training fold and use that median value to do the imputation both on the training and validation fold for a given CV split.

To do this we can prepare the features as previously but without the imputation: we just replace missing values by the -1 marker value:

In [ ]:
features = pd.concat([data.get(['Fare', 'Age']),
                      pd.get_dummies(data.Sex, prefix='Sex'),
                      pd.get_dummies(data.Pclass, prefix='Pclass'),
                      pd.get_dummies(data.Embarked, prefix='Embarked')],
                     axis=1)
features = features.drop('Sex_male', 1)

# Because of the following bug we cannot use NaN as the missing
# value marker, use a negative value as marker instead:
# https://github.com/scikit-learn/scikit-learn/issues/3044
features = features.fillna(-1)
features.head(5)

We can now use the `Imputer` transformer of scikit-learn to find the median value on the training set and apply it on missing values of both the training set and the test set.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features.values, target, random_state=0)

In [ ]:
from sklearn.preprocessing import Imputer

imputer = Imputer(strategy='median', missing_values=-1)

imputer.fit(X_train)

The median age computed on the training set is stored in the `statistics_` attribute.

In [ ]:
imputer.statistics_

Imputation can now happen by calling  the transform method:

In [ ]:
X_train_imputed = imputer.transform(X_train)
X_test_imputed = imputer.transform(X_test)

In [ ]:
np.any(X_train == -1)

In [ ]:
np.any(X_train_imputed == -1)

In [ ]:
np.any(X_test == -1)

In [ ]:
np.any(X_test_imputed == -1)

We can now use a pipeline that wraps an imputer transformer and the classifier itself:

In [ ]:
from sklearn.pipeline import Pipeline

imputer = Imputer(strategy='median', missing_values=-1)

classifier = LogisticRegression(C=1.)

pipeline = Pipeline([
    ('imp', imputer),
    ('clf', classifier),
])

scores = cross_val_score(pipeline, features.values, target, cv=5, n_jobs=4,
                         scoring='accuracy', )
print(scores.min(), scores.mean(), scores.max())

The mean cross-validation is slightly lower than we used the imputation on the whole data as we did earlier although not by much. This means that in this case the data-snooping was not really helping the model cheat by much.

### Parameter tuning with grid search

Note that thanks to the pipeline structure we can optimize the interaction of the imputation method with the parameters of the downstream classifier without cheating:

In [ ]:
%%time

from sklearn.model_selection import GridSearchCV

params = {
    'imp__strategy': ['mean', 'median'],
    'clf__C': [0.01, 0.1, 1., 10., 100., 1e3],
}
gs = GridSearchCV(pipeline, params, cv=5, scoring='roc_auc', n_jobs=4)
gs.fit(X_train, y_train)

In [ ]:
sorted(gs.grid_scores_, key=lambda x: x.mean_validation_score, reverse=True)

In [ ]:
gs.best_score_

In [ ]:
gs.best_params_

From this search we can conclude that the imputation by the 'mean' strategy is generally a slightly better imputation strategy when training a logistic regression model on this data.

## Further integrating scikit-learn and pandas

Helper tool for better sklearn / pandas integration: https://github.com/paulgb/sklearn-pandas by making it possible to embed the feature construction from the raw dataframe directly inside a pipeline.

### Credits

Thanks to:

- Kaggle for setting up the Titanic challenge.

- Alexandre Gramfort for being himself

- This blog post by Philippe Adjiman for inspiration:

http://www.philippeadjiman.com/blog/2013/09/12/a-data-science-exploration-from-the-titanic-in-r/